In [4]:
import warnings
import numpy as np
import pandas as pd
import seaborn as sns
import plotly_express as px
from pathlib import Path
from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import roc_auc_score
from catboost import CatBoostClassifier
from catboost import cv
from catboost import Pool
from matplotlib import pyplot as plt
from sklearn.preprocessing import scale


from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)
import plotly.graph_objs as go


%matplotlib inline

HUI_V_SRAKU bestTest = 0.8114043093

- DepTime_cat  24 категорий, удалены хвосты
- DepTime_cat - 24 категорий, удалены хвосты
- Флаги на месяцах
- Dep_minute
- DepTimeLog
- DepTimepairs - как категориальный фич
- DepTimesqrt
---


In [19]:
train_df_fd = pd.read_csv('flight_delays_train.csv')
test_df_fd = pd.read_csv('flight_delays_test.csv')
train_df_fd['flight'] = train_df_fd['Origin'] + '-->' + train_df_fd['Dest']
test_df_fd['flight'] = test_df_fd['Origin'] + '-->' + test_df_fd['Dest']
train_df_fd['dep_delayed_15min'] = train_df_fd['dep_delayed_15min'].map({'Y' : 1, 'N' : 0})
categ_feat_idx = np.where(train_df_fd.drop('dep_delayed_15min', axis=1).dtypes == 'object')[0]

train_df_fd.Month.replace(['c-%d' % i for i in range(1, 13)], list(range(1, 13)), inplace=True)
test_df_fd.Month.replace(['c-%d' % i for i in range(1, 13)], list(range(1, 13)), inplace=True)
train_df_fd.DayofMonth.replace(['c-%d' % i for i in range(1, 32)], list(range(1, 32)), inplace=True)
test_df_fd.DayofMonth.replace(['c-%d' % i for i in range(1, 32)], list(range(1, 32)), inplace=True)
train_df_fd.DayOfWeek.replace(['c-%d' % i for i in range(1, 8)], list(range(1, 8)), inplace=True)
test_df_fd.DayOfWeek.replace(['c-%d' % i for i in range(1, 8)], list(range(1, 8)), inplace=True)

In [20]:
# train_df_fd['DepTime'] = scale(train_df_fd['DepTime'])
# train_df_fd['Distance'] = scale(train_df_fd['Distance'])
# test_df_fd['Distance'] = scale(test_df_fd['Distance'])
# test_df_fd['DepTime'] = scale(test_df_fd['DepTime'])

In [21]:
# Дистанция и время отправления
train_df_fd['DepTime_cat'] = np.floor(train_df_fd.DepTime / 100).astype('int')
test_df_fd['DepTime_cat'] = np.floor(test_df_fd.DepTime / 100).astype('int')
train_df_fd = train_df_fd[(train_df_fd['DepTime_cat'] > 4) & (train_df_fd['DepTime_cat'] < 24)]
train_df_fd['distance_cat'] = np.floor(train_df_fd.Distance / 200).astype('int')
test_df_fd['distance_cat'] = np.floor(test_df_fd.Distance / 200).astype('int')
train_df_fd = train_df_fd[train_df_fd['distance_cat'] < 15]

# Дни и месяцы

train_df_fd['Months [6,7,12]'] = train_df_fd['Month'].apply(lambda x: 1 if x >= 6 \
                                                         and x <= 7 or x == 12 else 0)
test_df_fd['Months [6,7,12]'] = test_df_fd['Month'].apply(lambda x: 1 if x >= 6 \
                                                       and x <= 7  or x == 12 else 0)
# train_df_fd['Months [2,3]'] = train_df_fd['Month'].apply(lambda x: 1 if x >= 2 \
#                                                          and x <= 3 else 0)
# test_df_fd['Months [2,3]'] = test_df_fd['Month'].apply(lambda x: 1 if x >= 2 \
#                                                        and x <= 3 else 0)

# Еще признаки
train_df_fd['Dep_minute'] =  train_df_fd['DepTime']%100
test_df_fd['Dep_minute'] =  test_df_fd['DepTime']%100
train_df_fd['DepTimeLog'] = train_df_fd['DepTime'].apply(lambda x: np.log(x))
test_df_fd['DepTimeLog'] = test_df_fd['DepTime'].apply(lambda x: np.log(x))
train_df_fd['DepTimepairs'] = train_df_fd['DepTime'] * np.append(train_df_fd['DepTime'][1:], 1)
test_df_fd['DepTimepairs'] = test_df_fd['DepTime'] * np.append(test_df_fd['DepTime'][1:], 1)
train_df_fd['DepTimesqrt'] = train_df_fd['DepTime'].apply(lambda x: np.sqrt(x))
test_df_fd['DepTimesqrt'] = test_df_fd['DepTime'].apply(lambda x: np.sqrt(x))

In [22]:
y_train = train_df_fd['dep_delayed_15min']
train_df_fd = train_df_fd.drop('dep_delayed_15min', axis=1)

In [23]:
X_train_part, X_valid, y_train_part, y_valid = train_test_split(train_df_fd, y_train, 
                                                                test_size=0.3, 
                                                                random_state=17)
len(X_train_part.columns), X_train_part.columns, np.append(categ_feat_idx, [7,8,9])

(16, Index(['Month', 'DayofMonth', 'DayOfWeek', 'DepTime', 'UniqueCarrier',
        'Origin', 'Dest', 'Distance', 'flight', 'DepTime_cat', 'distance_cat',
        'Months [6,7,12]', 'Dep_minute', 'DepTimeLog', 'DepTimepairs',
        'DepTimesqrt'],
       dtype='object'), array([0, 1, 2, 4, 5, 6, 8, 7, 8, 9], dtype=int64))

In [3]:
# %%time
ctb = CatBoostClassifier(random_seed=17, eval_metric = 'AUC')
ctb.fit(X_train_part, y_train_part,
        eval_set=(X_valid, y_valid),
        cat_features=np.append(categ_feat_idx, [9, 10, 15]),
        early_stopping_rounds = 500,
        verbose = 200,
        use_best_model=True,
        plot=False,
    
       );


NameError: name 'CatBoostClassifier' is not defined

In [26]:
ctb.get_feature_importance(prettified=True)

,Feature Id,Importances
0,DepTime_cat,14.318910
1,Dep_minute,12.462822
2,flight,11.942588
3,UniqueCarrier,10.036473
4,Origin,8.121630
5,Dest,7.457263
6,DepTime,6.579410
7,Distance,5.268970
8,DayofMonth,4.537578
9,Month,4.270797


In [27]:
# %%time
ctb = CatBoostClassifier(random_seed=17, eval_metric = 'AUC')
ctb.fit(X_train_part, y_train_part,
        eval_set=(X_valid, y_valid),
        cat_features=np.append(categ_feat_idx, [9, 10, 15]),
        early_stopping_rounds = 500,
        verbose = 200,
        use_best_model=True,
        plot=False,
    
       );

CatBoostError: Invalid type for cat_feature[non-default value idx=0,feature_idx=15]=28.319604517012593 : cat_features must be integer or string, real number values and NaN values should be converted to string.

In [ ]:
ctb.get_feature_importance(prettified=True)

In [24]:
params = {'loss_function':'Logloss',
          'eval_metric':'AUC',
          'verbose': 200,
          'random_seed': 17
         }
all_train_data = Pool(data=train_df_fd,
                      label=y_train,
                      cat_features=np.append(categ_feat_idx, [9, 10, 14])
                     )
scores = cv(pool=all_train_data,
            params=params, 
            fold_count=3,
            seed=17, 
            shuffle=True,
            stratified=True, # if True the folds are made by preserving the percentage of samples for each class
            plot=False
           )





0:	test: 0.6646844	best: 0.6646844 (0)	total: 1.44s	remaining: 24m
200:	test: 0.7641324	best: 0.7641324 (200)	total: 4m 17s	remaining: 17m 4s
400:	test: 0.7734746	best: 0.7734746 (400)	total: 9m 24s	remaining: 14m 2s
600:	test: 0.7818712	best: 0.7818712 (600)	total: 13m 43s	remaining: 9m 6s
800:	test: 0.7871238	best: 0.7871238 (800)	total: 17m 51s	remaining: 4m 26s
999:	test: 0.7910532	best: 0.7910532 (999)	total: 21m 53s	remaining: 0us
